<a href="https://colab.research.google.com/github/Pusse-01/Crop-Disease-Alerting-System/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [70]:
data = pd.read_excel('/content/drive/MyDrive/plant disease detection/dataset.xlsx')

In [71]:
data.head()

,Crop,Family,Temp - C,RainFall - mm,PH val,Zone,Season
0,BigOnion,Amaryllidaceae,10.0-25.0,<750,6.0-8.0,dry,yala
1,Chili,Solanaceae,20.0-30.0,600-1000,6.0-7.0,"dry,wet,intermediate","yala,maha"
2,RedOnion,Amaryllidaceae,15.0-20.0,650-750,6.0-7.0,"dry, intermediate",yala
3,Corn,Poaceae,25.0-30.0,600-1000,6.0-7.0,"dry,intermediate",yala
4,FingerMillet,Poaceae,15.0-30.0,800-1000,5.0-8.0,"dry,intermediate","yala,maha"


In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Crop           108 non-null    object
 1   Family         108 non-null    object
 2   Temp - C       103 non-null    object
 3   RainFall - mm  76 non-null     object
 4   PH val         102 non-null    object
 5   Zone           108 non-null    object
 6   Season         108 non-null    object
dtypes: object(7)
memory usage: 6.0+ KB


In [73]:
data.isnull().sum()

Crop              0
Family            0
Temp - C          5
RainFall - mm    32
PH val            6
Zone              0
Season            0
dtype: int64

In [74]:
data.drop(['RainFall - mm'], axis=1, inplace=True)

In [75]:
data = data.dropna()

In [76]:
data.describe()

,Crop,Family,Temp - C,PH val,Zone,Season
count,101,101,101,101,101,101
unique,101,44,23,12,14,4
top,BigOnion,Solanaceae,25.0-30.0,6.0-7.0,"dry,intermediate","yala,maha"
freq,1,15,21,33,26,84


In [77]:
data['Crop'].unique()

array(['BigOnion', 'Chili', 'RedOnion', 'Corn', 'FingerMillet', 'SoyBean',
       'BlackGram', 'Kaupi', 'Pigeon Pea', 'Potato', 'Cassava',
       'SweetPotato', 'Yams', 'GroundNut', 'Sesame', 'Mustard',
       'SunFlower', 'Cabbage', 'Radish', 'Carrot', 'Tomato', 'Capsicum',
       'Brinjal', 'Okra', 'Beans', 'WingedBean', 'GreenYardLongBean',
       'RedYardLongBean', 'Pumpkin', 'Cucumber', 'SnakeGourd', 'Luffa',
       'BitterGourd', 'BeetRoot', 'AmericanOyster', 'Leeks', 'Elabatu',
       'Thibbatu', 'SpineGourd', 'Lettuce', 'IceBurg', 'KnolKhol',
       'CauliFlower', 'Gotukola', 'Mukunuwenna', 'Kathurumurunga',
       'KangKung', 'Abalone', 'PinkOyster', 'Mango', 'Pineapple',
       'Avocado', 'Banana', 'WaterMelon', 'Jamanaran', 'Duriyan', 'Papaw',
       'Guava', 'Rambutan', 'Grapes', 'Woodapple', 'Beli', 'Mangosteen',
       'Pomegranate', 'DragonFruit', 'PassionFruit', 'Anoda', 'Ugurassa',
       'StarFruit', 'Amla', 'Lawulu', 'Vanila', 'Turmeric', 'Papper',
       'Nutmeg', '

In [78]:
len(data['Crop'].unique())

101

In [79]:
data['Family'].unique()

array(['Amaryllidaceae', 'Solanaceae', 'Poaceae', 'Fabaceae',
       'Pedaliaceae', 'Brassicaceae', 'Asteraceae', 'Apiaceca',
       'Malvaceca', 'Cucurbitaceae', 'Chenopodiaceae', 'Agaricaceae',
       'Alliaceae', 'Amaranthaceae', 'Convolvulaceae', 'Anacardiaceae',
       'Bromiliceae', 'Lauraceae', 'Musaceae', 'Rutaceae', 'Bombacaceae',
       'Caricaceae', 'Myrtaceae', 'Sapindaceae', 'Vitaceae', 'Clusiaceae',
       'Punicaceae', 'Cactaceae', 'Passifloraciae', 'Annonaceae',
       'Flacourtiaceae', 'Oxaliadaceae', 'Euphorbiaceae', 'Sapotaceae',
       'Orchidaceae', 'Zingiberaceae', 'Piperaceae', 'Myristiccaceae',
       'Rubiaceae', 'Sterculiaceae', 'Mertaceae', 'Palmae', 'Graminae',
       'Moraceae'], dtype=object)

In [80]:
data['Season'].unique()

array(['yala', 'yala,maha', 'yala, maha', 'maha'], dtype=object)

In [81]:
data["Season"].replace({"yala": 0, "yala, maha": 2, "yala,maha": 2 ,"maha": 1}, inplace=True)
data.head()

,Crop,Family,Temp - C,PH val,Zone,Season
0,BigOnion,Amaryllidaceae,10.0-25.0,6.0-8.0,dry,0
1,Chili,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate",2
2,RedOnion,Amaryllidaceae,15.0-20.0,6.0-7.0,"dry, intermediate",0
3,Corn,Poaceae,25.0-30.0,6.0-7.0,"dry,intermediate",0
4,FingerMillet,Poaceae,15.0-30.0,5.0-8.0,"dry,intermediate",2


In [82]:
data['Season'].unique()

array([0, 2, 1])

In [84]:
data['Zone'].unique()

array(['dry', 'dry,wet,intermediate', 'dry, intermediate',
       'dry,intermediate', 'wet, dry', 'dry,wet', 'dry,intermediate,wet',
       'wet', 'intermediate,wet', 'wet,intermediate',
       'wet,intermediate,dry', 'intermediate', 'wet, intermediate',
       'wet,dry'], dtype=object)

In [85]:
data["Zone"].replace({"dry": 0, "wet": 1, "intermediate": 2, "dry,wet": 3,"wet,dry":3, "dry, intermediate": 4, "dry,intermediate": 4,
                        "wet, dry": 3, "dry,wet,intermediate": 6, "dry,intermediate,wet": 6, "intermediate,wet":5,
                        "wet,intermediate":5, "wet,intermediate,dry":6,"wet, intermediate":5,}, inplace=True)
data.head()

,Crop,Family,Temp - C,PH val,Zone,Season
0,BigOnion,Amaryllidaceae,10.0-25.0,6.0-8.0,0,0
1,Chili,Solanaceae,20.0-30.0,6.0-7.0,6,2
2,RedOnion,Amaryllidaceae,15.0-20.0,6.0-7.0,4,0
3,Corn,Poaceae,25.0-30.0,6.0-7.0,4,0
4,FingerMillet,Poaceae,15.0-30.0,5.0-8.0,4,2


In [86]:
data['Zone'].unique()

array([0, 6, 4, 3, 1, 5, 2])

In [87]:
data['PH val'].unique()

array(['6.0-8.0', '6.0-7.0', '5.0-8.0', '6.0-6.5', '5.0-7.0', '5.0-6.0',
       '6.0-9.0', '5.0-10.0', '6.0-10.0', '4.0-6.0', '<7.0', '4.0-9.0'],
      dtype=object)

In [88]:
data['Temp - C'].unique()

array(['10.0-25.0', '20.0-30.0', '15.0-20.0', '25.0-30.0', '15.0-30.0',
       '20.0-35.0', '<24', '20.0-40.0', '>20', '10.0-15.0', '20.0-25.0',
       '<30', '<20', '15.0-25.0', '>27', '15.0-40.0', '10.0-40.0',
       '30.0-35.0', '25.0-40.0', '25.0-35.0', '10.0-30.0', '10.0-20.0',
       '<46'], dtype=object)

In [89]:
labels = data['Family'].astype('category').cat.categories.tolist()
data['Family'] = data['Family'].astype('category').cat.codes
replaced_data = {'Family' : {k: v for k,v in zip(labels,list(range(0,len(labels))))}}

print(replaced_data)

{'Family': {'Agaricaceae': 0, 'Alliaceae': 1, 'Amaranthaceae': 2, 'Amaryllidaceae': 3, 'Anacardiaceae': 4, 'Annonaceae': 5, 'Apiaceca': 6, 'Asteraceae': 7, 'Bombacaceae': 8, 'Brassicaceae': 9, 'Bromiliceae': 10, 'Cactaceae': 11, 'Caricaceae': 12, 'Chenopodiaceae': 13, 'Clusiaceae': 14, 'Convolvulaceae': 15, 'Cucurbitaceae': 16, 'Euphorbiaceae': 17, 'Fabaceae': 18, 'Flacourtiaceae': 19, 'Graminae': 20, 'Lauraceae': 21, 'Malvaceca': 22, 'Mertaceae': 23, 'Moraceae': 24, 'Musaceae': 25, 'Myristiccaceae': 26, 'Myrtaceae': 27, 'Orchidaceae': 28, 'Oxaliadaceae': 29, 'Palmae': 30, 'Passifloraciae': 31, 'Pedaliaceae': 32, 'Piperaceae': 33, 'Poaceae': 34, 'Punicaceae': 35, 'Rubiaceae': 36, 'Rutaceae': 37, 'Sapindaceae': 38, 'Sapotaceae': 39, 'Solanaceae': 40, 'Sterculiaceae': 41, 'Vitaceae': 42, 'Zingiberaceae': 43}}


In [91]:
data.rename(columns = {'Temp - C':'temp_C', 'PH val':'pH_val'}, inplace = True)

In [92]:
labels = data['temp_C'].astype('category').cat.categories.tolist()
data['temp_C'] = data['temp_C'].astype('category').cat.codes
replaced_data = {'Temp_C' : {k: v for k,v in zip(labels,list(range(0,len(labels))))}}

print(replaced_data)

{'Temp_C': {'10.0-15.0': 0, '10.0-20.0': 1, '10.0-25.0': 2, '10.0-30.0': 3, '10.0-40.0': 4, '15.0-20.0': 5, '15.0-25.0': 6, '15.0-30.0': 7, '15.0-40.0': 8, '20.0-25.0': 9, '20.0-30.0': 10, '20.0-35.0': 11, '20.0-40.0': 12, '25.0-30.0': 13, '25.0-35.0': 14, '25.0-40.0': 15, '30.0-35.0': 16, '<20': 17, '<24': 18, '<30': 19, '<46': 20, '>20': 21, '>27': 22}}


In [93]:
labels = data['pH_val'].astype('category').cat.categories.tolist()
data['pH_val'] = data['pH_val'].astype('category').cat.codes
replaced_data = {'pH_val' : {k: v for k,v in zip(labels,list(range(0,len(labels))))}}

print(replaced_data)

{'pH_val': {'4.0-6.0': 0, '4.0-9.0': 1, '5.0-10.0': 2, '5.0-6.0': 3, '5.0-7.0': 4, '5.0-8.0': 5, '6.0-10.0': 6, '6.0-6.5': 7, '6.0-7.0': 8, '6.0-8.0': 9, '6.0-9.0': 10, '<7.0': 11}}


In [94]:
data.drop(['Crop'], axis=1, inplace=True)

In [95]:
data.head()

,Family,temp_C,pH_val,Zone,Season
0,3,2,9,0,0
1,40,10,8,6,2
2,3,5,8,4,0
3,34,13,8,4,0
4,34,7,5,4,2


In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 107
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Family  101 non-null    int8 
 1   temp_C  101 non-null    int8 
 2   pH_val  101 non-null    int8 
 3   Zone    101 non-null    int64
 4   Season  101 non-null    int64
dtypes: int64(2), int8(3)
memory usage: 2.7 KB


In [97]:
import plotly.express as px
fig = px.scatter_matrix(data,
width=1200, height=1600)
fig.show()

In [98]:
X=data
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

In [99]:
kmeans = KMeans(
        n_clusters=6, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
kmeans.fit(X)
clusters=pd.DataFrame(X,columns=data.columns)
clusters['label']=kmeans.labels_
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig4.show()

In [100]:
pie=clusters.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label',color=['blue','red','green','yellow','grey','brown'])

In [101]:
cols = data.columns

In [102]:
ms = MinMaxScaler()
X = ms.fit_transform(data)

In [103]:
X = pd.DataFrame(X, columns=[cols])

In [104]:
X.head()

,Family,temp_C,pH_val,Zone,Season
0,0.069767,0.090909,0.818182,0.000000,0.0
1,0.930233,0.454545,0.727273,1.000000,1.0
2,0.069767,0.227273,0.727273,0.666667,0.0
3,0.790698,0.590909,0.727273,0.666667,0.0
4,0.790698,0.318182,0.454545,0.666667,1.0


In [105]:
kmeans.cluster_centers_

array([[ 3.60000000e-01,  6.29090909e-01,  4.87272727e-01,
         8.00000000e-01,  9.80000000e-01],
       [ 8.00000000e-01,  3.98181818e-01,  2.87272727e-01,
         7.26666667e-01,  9.60000000e-01],
       [ 5.60253700e-01,  4.17355372e-01,  6.28099174e-01,
         5.45454545e-01, -1.11022302e-16],
       [ 3.27695560e-01,  5.45454545e-01,  6.94214876e-01,
         1.81818182e-01,  9.09090909e-01],
       [ 8.56887299e-01,  4.47552448e-01,  7.48251748e-01,
         9.48717949e-01,  1.00000000e+00],
       [ 1.59883721e-01,  2.55681818e-01,  6.47727273e-01,
         9.37500000e-01,  1.00000000e+00]])

In [112]:
predicted_labels = kmeans.labels_

In [107]:
sample_test = np.array([0.088235,0.08, 0.90625, 0.000000,0.0])
second_test=sample_test.reshape(1, -1)
kmeans.predict(second_test)

array([2], dtype=int32)

In [108]:
data['Family'].unique()

array([ 3, 40, 34, 18, 32,  9,  7,  6, 22, 16, 13,  0,  1,  2, 15,  4, 10,
       21, 25, 37,  8, 12, 27, 38, 42, 14, 35, 11, 31,  5, 19, 29, 17, 39,
       28, 43, 33, 26, 36, 41, 23, 30, 20, 24], dtype=int8)

In [109]:
data['temp_C'].unique()

array([ 2, 10,  5, 13,  7, 11, 18, 12, 21,  0,  9, 19, 17,  6, 22,  8,  4,
       16, 15, 14,  3,  1, 20], dtype=int8)

In [110]:
data['pH_val'].unique()

array([ 9,  8,  5,  7,  4,  3, 10,  2,  6,  0, 11,  1], dtype=int8)

In [111]:
X= [40, 10, 8, 6, 2]
sample_test = np.array(X)
second_test=sample_test.reshape(1, -1)
kmeans.predict(second_test)


array([4], dtype=int32)

In [44]:
# save the model
import pickle
pickle.dump(kmeans, open("/content/drive/MyDrive/plant disease detection/model.pkl", "wb"))

In [45]:
# load the model
model = pickle.load(open("/content/drive/MyDrive/plant disease detection/model.pkl", "rb"))

In [46]:
model.predict(second_test)

array([4], dtype=int32)

In [121]:
original_data = pd.read_excel('/content/drive/MyDrive/plant disease detection/dataset.xlsx')

In [122]:
original_data.drop(['RainFall - mm'], axis=1, inplace=True)
original_data = original_data.dropna()
original_data.head()

,Crop,Family,Temp - C,PH val,Zone,Season
0,BigOnion,Amaryllidaceae,10.0-25.0,6.0-8.0,dry,yala
1,Chili,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate","yala,maha"
2,RedOnion,Amaryllidaceae,15.0-20.0,6.0-7.0,"dry, intermediate",yala
3,Corn,Poaceae,25.0-30.0,6.0-7.0,"dry,intermediate",yala
4,FingerMillet,Poaceae,15.0-30.0,5.0-8.0,"dry,intermediate","yala,maha"


In [123]:
original_data['Predicted_category'] = predicted_labels

In [124]:
original_data.head()

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
0,BigOnion,Amaryllidaceae,10.0-25.0,6.0-8.0,dry,yala,2
1,Chili,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate","yala,maha",4
2,RedOnion,Amaryllidaceae,15.0-20.0,6.0-7.0,"dry, intermediate",yala,2
3,Corn,Poaceae,25.0-30.0,6.0-7.0,"dry,intermediate",yala,2
4,FingerMillet,Poaceae,15.0-30.0,5.0-8.0,"dry,intermediate","yala,maha",1


In [125]:
original_data.to_csv('/content/drive/MyDrive/plant disease detection/predicted_outputs.csv', index=False)

In [133]:
print(original_data['Crop'][1])

Chili


In [141]:
original_data['Predicted_category'].unique()

array([2, 4, 1, 0, 5, 3], dtype=int32)

In [144]:
original_data.loc[original_data['Predicted_category'] == 0]

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
6,SoyBean,Fabaceae,25.0-30.0,6.0-6.5,"dry,intermediate","yala,maha",0
7,BlackGram,Fabaceae,20.0-35.0,6.0-7.0,"dry, intermediate","yala,maha",0
16,GroundNut,Fabaceae,25.0-30.0,6.0-7.0,"dry,intermediate,wet","yala,maha",0
26,Okra,Malvaceca,20.0-30.0,6.0-7.0,"dry,intermediate","yala,maha",0
27,Beans,Fabaceae,<30,5.0-7.0,"intermediate,wet","yala,maha",0
29,GreenYardLongBean,Fabaceae,20.0-25.0,5.0-7.0,"dry,intermediate,wet","yala,maha",0
30,RedYardLongBean,Fabaceae,20.0-25.0,5.0-7.0,"dry,intermediate,wet","yala,maha",0
34,Luffa,Cucurbitaceae,25.0-30.0,5.0-8.0,"dry,wet,intermediate","yala,maha",0
35,BitterGourd,Cucurbitaceae,25.0-30.0,5.0-8.0,"dry,intermediate","yala,maha",0
41,SpineGourd,Cucurbitaceae,25.0-30.0,5.0-8.0,"dry,intermediate","yala,maha",0


In [145]:
original_data.loc[original_data['Predicted_category'] == 1]

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
4,FingerMillet,Poaceae,15.0-30.0,5.0-8.0,"dry,intermediate","yala,maha",1
12,Cassava,Solanaceae,25.0-30.0,5.0-6.0,"dry,wet","yala,maha",1
13,SweetPotato,Solanaceae,20.0-40.0,5.0-6.0,"dry,wet,intermediate","yala, maha",1
15,Yams,Solanaceae,25.0-30.0,5.0-6.0,"wet, dry","yala,maha",1
17,Sesame,Pedaliaceae,25.0-30.0,5.0-8.0,"dry,intermediate","yala,maha",1
24,Capsicum,Solanaceae,15.0-20.0,5.0-7.0,"dry,intermediate,wet","yala,maha",1
25,Brinjal,Solanaceae,20.0-25.0,5.0-6.0,"dry,intermediate","yala,maha",1
39,Elabatu,Solanaceae,20.0-25.0,5.0-6.0,"dry,intermediate","yala,maha",1
40,Thibbatu,Solanaceae,20.0-25.0,5.0-6.0,"dry,intermediate","yala,maha",1
56,Banana,Musaceae,15.0-40.0,5.0-7.0,"dry,intermediate,wet","yala,maha",1


In [146]:
original_data.loc[original_data['Predicted_category'] == 2]

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
0,BigOnion,Amaryllidaceae,10.0-25.0,6.0-8.0,dry,yala,2
2,RedOnion,Amaryllidaceae,15.0-20.0,6.0-7.0,"dry, intermediate",yala,2
3,Corn,Poaceae,25.0-30.0,6.0-7.0,"dry,intermediate",yala,2
8,Kaupi,Fabaceae,20.0-35.0,6.0-7.0,"dry,intermediate",yala,2
9,Pigeon Pea,Fabaceae,15.0-20.0,5.0-8.0,"dry, intermediate",yala,2
10,Potato,Solanaceae,<24,5.0-7.0,"wet, dry",yala,2
31,Pumpkin,Cucurbitaceae,15.0-30.0,5.0-8.0,"dry,intermediate",yala,2
63,Grapes,Vitaceae,25.0-30.0,6.0-8.0,dry,yala,2
77,Turmeric,Zingiberaceae,20.0-35.0,5.0-7.0,"wet,intermediate",yala,2
92,Breadfruit,Moraceae,15.0-40.0,6.0-7.0,"dry,intermediate",yala,2


In [147]:
original_data.loc[original_data['Predicted_category'] == 3]

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
19,SunFlower,Asteraceae,>20,6.0-9.0,dry,"yala,maha",3
20,Cabbage,Brassicaceae,15.0-20.0,6.0-7.0,wet,maha,3
32,Cucumber,Cucurbitaceae,20.0-30.0,6.0-7.0,"dry,wet","yala,maha",3
33,SnakeGourd,Cucurbitaceae,20.0-30.0,5.0-8.0,dry,"yala,maha",3
42,Lettuce,Asteraceae,<20,6.0-7.0,wet,"yala,maha",3
43,IceBurg,Asteraceae,<20,6.0-7.0,wet,"yala,maha",3
46,CauliFlower,Brassicaceae,15.0-20.0,6.0-7.0,wet,"yala,maha",3
76,Vanila,Orchidaceae,20.0-30.0,5.0-8.0,wet,"yala,maha",3
79,Nutmeg,Myristiccaceae,20.0-30.0,6.0-7.0,intermediate,"yala,maha",3
84,Clove,Mertaceae,20.0-30.0,6.0-7.0,wet,maha,3


In [148]:
original_data.loc[original_data['Predicted_category'] == 4]

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
1,Chili,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate","yala,maha",4
23,Tomato,Solanaceae,20.0-25.0,5.0-8.0,"dry,wet,intermediate","yala,maha",4
64,Woodapple,Rutaceae,25.0-40.0,6.0-9.0,"wet,intermediate,dry","yala,maha",4
69,PassionFruit,Passifloraciae,20.0-30.0,6.0-8.0,"dry,intermediate,wet","yala,maha",4
75,Lawulu,Sapotaceae,10.0-40.0,6.0-7.0,"wet,intermediate","yala,maha",4
89,Areca Nut,Palmae,25.0-40.0,<7.0,"wet,intermediate","yala,maha",4
91,Ash plantain,Musaceae,15.0-40.0,6.0-7.0,"dry,intermediate,wet","yala,maha",4
97,Green Bell Pepper,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate","yala,maha",4
98,Red Bell Pepper,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate","yala,maha",4
99,Yellow Bell Pepper,Solanaceae,20.0-30.0,6.0-7.0,"dry,wet,intermediate","yala,maha",4


In [149]:
original_data.loc[original_data['Predicted_category'] == 5]

,Crop,Family,Temp - C,PH val,Zone,Season,Predicted_category
18,Mustard,Brassicaceae,15.0-30.0,6.0-8.0,"dry,intermediate","yala,maha",5
21,Radish,Brassicaceae,10.0-15.0,6.0-8.0,"dry,wet,intermediate","yala,maha",5
22,Carrot,Apiaceca,15.0-20.0,6.0-7.0,"dry,wet,intermediate","yala,maha",5
28,WingedBean,Fabaceae,15.0-30.0,5.0-8.0,"dry,intermediate,wet","yala,maha",5
36,BeetRoot,Chenopodiaceae,20.0-30.0,6.0-8.0,"dry,wet,intermediate","yala,maha",5
37,AmericanOyster,Agaricaceae,15.0-20.0,5.0-7.0,"dry,wet,intermediate","yala,maha",5
38,Leeks,Alliaceae,10.0-25.0,5.0-6.0,"wet,intermediate","yala,maha",5
45,KnolKhol,Brassicaceae,15.0-25.0,6.0-8.0,"dry,wet,intermediate","yala,maha",5
47,Gotukola,Apiaceca,15.0-30.0,6.0-7.0,"intermediate,wet","yala,maha",5
48,Mukunuwenna,Amaranthaceae,15.0-30.0,6.0-7.0,"intermediate,wet","yala,maha",5
